# Goals

1. Predict the customer spending for the fourth quarter of the current year.

2. Classify the Accounts eligible for **Credit Line Increase without Risk**, **Accounts eligible for Credit Line Increase but has risk of potential defaults or fraud**, **No Credit Line Increase required** and **Non-Performing accounts that pose a high risk**. 

3. For each of the account segments what would be a credit line increase that could be offered at individual account?

## Steps: 
1. Analyze historical spending patterns using account data and non-monetary information.
- Make plots showing amount spent by date
- Focus on time from October through December
2. Predict the customer spending for the fourth quarter of the current year. Specifically, Using the recent last
eight months of spending can we predict the customer spending for Q4 2025 (October to December
2025).
- Make a linear regression model using variables of interest
3. Classify accounts in to segments that can help identify potential accounts that would need a credit line
increase because of the predicted spending.
- Credit limit - Amount spent; if spent > 90% of limit 
4. Using risk factors overextension, fraud, or potential defaults, suggest the amount of credit line increase
that could be given to an account.



Questions to ask:
- Explain Open to Buy; 
- Random Number? 
- Old vs New Behavior Score?
Negative Current Utiliz? 


## Variables of Interest
1. Behavior Score Old/New -- (Account Level Features)
2. Payment History -- (Card Holder Account)
3. overlimit_type_flag -- (Card Holder Account)
4. External_status_reason_code -- (Card Holder Account)
5. Nsf_count ie nonsufficient funds count last 12 months -- (Account Level Features)
6. Account Card type (Dual or PLCC) -- (Card Holder Account)
7. OTB (Open to Buy, )
8. Credit Score 
9. Special Finance Charge? - Card Holder Account
10. Current Utiliz (current balance as percentage of current credit line)

## New Variables:
- Sum of total spending between 10/2024 - 12/2024
- Monthly sum of money spent 
- Calculate monthly credit limit

### For Predicting Q4 Spending
- Sum of total spending between 10/2024 - 12/2024
- Account Card Type


In [775]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import statsmodels.formula.api as smf 
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix

### Reading in Files

In [776]:
df_account_dim = pd.read_csv("/Users/kaitlinbaggot/Desktop/datathon 2025/account_dim_20250325.csv")
df_account_dim

,current_account_nbr,client_id,open_date,card_activation_date,card_activation_flag,ebill_ind,date_in_collection,overlimit_type_flag,payment_hist_1_12_mths,payment_hist_13_24_mths,employee_code,external_status_reason_code,special_finance_charge_ind,pscc_ind,ext_status_reason_cd_desc,account_card_type
0,nTgalbpd5Rsol7CA,YCTHAA278DC,2020-07-17T00:00:00.000-04:00,NaN,8,E,NaN,0,ZZZZZZIAZZZZ,ZZZZIIIQIIII,NaN,0,NaN,0,NaN,DUAL CARD
1,xYaLrD9IvmbVl1PB,CCSP489PLCC,2024-07-24T00:00:00.000-04:00,2024-08-09T00:00:00.000-04:00,0,E,NaN,0,IQIIIQIA,"\\\\\\""""",NaN,0,NaN,1,NaN,DUAL CARD
2,JM78WNziiRWgmL8e,HGBHARRFT984,2024-05-05T00:00:00.000-04:00,2024-05-05T00:00:00.000-04:00,0,"\\\\\\""""",NaN,0,IIIQIQIIIIA,"\\\\\\""""",NaN,0,NaN,0,NaN,PLCC
3,bv4NLVhWTGCf8KgC,OSELW263,2024-07-28T00:00:00.000-04:00,2024-09-08T00:00:00.000-04:00,0,"\\\\\\""""",NaN,0,I1III1A,"\\\\\\""""",NaN,0,NaN,0,NaN,PLCC
4,nrXceQabYIHo31lp,HGBHARRFT984,2024-09-15T00:00:00.000-04:00,2024-09-15T00:00:00.000-04:00,0,"\\\\\\""""",NaN,0,"\\\\\\""""","\\\\\\""""",NaN,0,NaN,0,NaN,PLCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18065,K8ixzIELRNtw1drn,YAUAPPLLDA934,2023-11-27T00:00:00.000-05:00,NaN,7,E,NaN,0,ZZZZZQQQQQQQ,QQ#A,NaN,35,NaN,0,A35 Temporary Freeze - Customer may have lost ...,DUAL CARD
18066,CRIi94oT7Dddh2zu,TJX513PLCC,2023-09-11T00:00:00.000-04:00,2023-09-11T00:00:00.000-04:00,0,"\\\\\\""""",NaN,0,ZZZZZZZZZZZZ,ZZZZIA,NaN,35,NaN,0,A35 Temporary Freeze - Customer may have lost ...,PLCC
18067,XiW0as5dJX6GJRi0,CCSP489PLCC,2023-12-03T00:00:00.000-05:00,NaN,7,E,NaN,0,ZZZIIIIIIIII,IIIA,NaN,35,NaN,1,A35 Temporary Freeze - Customer may have lost ...,DUAL CARD
18068,Xu1i4wqn776tFKU6,CMSSSNAO606,2023-07-27T00:00:00.000-04:00,2025-01-20T00:00:00.000-05:00,0,E,NaN,0,Z0ZZZZIIAIQQ,QQQIQIA,NaN,46,NaN,0,A46 Universal Fraud Team Potential Fraud,PLCC


In [777]:
df_transactions = pd.read_csv("/Users/kaitlinbaggot/Desktop/datathon 2025/transaction_fact_20250325.csv")
df_transactions

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt,product_amt,product_qty,...,first_purchase_ind,adj_orgn_tran_dt,curr_markup_fee,fcr_amount,fcr_flag,fcr_rate_of_exchange,frgn_curr_code,frgn_tran_amt,us_equiv_amt,posting_date
0,X7jfKh6xrPAB8Tx6,SALE,253,2024-06-05,15.78,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,15.78,0.0,2024-06-06
1,yntD77AZDylS48Q4,SALE,253,2024-06-19,14.85,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,14.85,0.0,2024-06-20
2,LIJPI0sK28Pa7fX2,SALE,253,2024-06-26,136.16,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,136.16,0.0,2024-06-27
3,CMAr5Apxwdzpvoze,SALE,253,2024-08-15,8.74,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,8.74,0.0,2024-08-16
4,eJSfTCGPvJulGzd3,SALE,253,2024-08-17,26.65,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,26.65,0.0,2024-08-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493331,ktpdvlbVRuywp2Po,SALE,253,2025-03-07,3.99,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,3.99,0.0,2025-03-07
493332,xwgOt4lvAHr517sa,SALE,253,2024-07-27,6.13,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,6.13,0.0,2024-07-28
493333,zp60CmOQNComnHDT,SALE,253,2025-02-04,15.62,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,15.62,0.0,2025-02-05
493334,8M5fnshTsgW9mW8Y,PAYMENT,271,2025-02-05,100.00,NaN,0,0,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,"\""\""",0.00,0.0,2025-02-05


In [778]:
df_rams = pd.read_csv("/Users/kaitlinbaggot/Desktop/datathon 2025/rams_batch_cur_20250325.csv")
df_rams

,cu_bhv_scr,ca_cash_bal_pct_crd_line,ca_cash_bal_pct_cash_line,cu_nbr_days_dlq,cu_nbr_of_plastics,ca_avg_utilz_lst_6_mnths,cu_cash_line_am,cu_crd_bureau_scr,cu_crd_line,cu_next_crd_line_rev_date,...,rb_new_bhv_scr,rb_crd_gr_new_crd_gr,cu_processing_date,mo_tot_sales_array_1,mo_tot_sales_array_2,mo_tot_sales_array_3,mo_tot_sales_array_4,mo_tot_sales_array_5,mo_tot_sales_array_6,ca_avg_utilz_lst_3_mnths
0,779,0,0,0,2,2,5000,782,25000,0,...,776,R,2024-12-17,997.56,521.07,647.86,1045.03,858.49,334.54,2
1,735,0,999,0,1,2,0,836,1500,0,...,735,R,2025-02-21,0.00,37.65,60.25,175.67,54.08,0.00,0
2,745,0,0,0,1,2,8000,832,40000,0,...,748,R,2024-10-10,1024.55,578.80,665.81,816.43,2526.27,3129.23,1
3,13,0,0,0,1,0,60,684,300,0,...,13,O,2025-01-27,0.00,0.00,0.00,0.00,0.00,0.00,0
4,6,0,999,0,1,7,0,681,5200,0,...,6,P,2025-03-19,895.33,0.00,0.00,0.00,0.00,0.00,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96794,6,0,0,30,1,15,90,711,900,0,...,628,K,2024-11-14,0.00,185.10,0.00,0.00,0.00,0.00,15
96795,709,0,0,0,1,8,999999999999999,833,6700,0,...,703,R,2025-02-27,0.00,0.00,0.00,0.00,0.00,0.00,7
96796,755,0,0,0,1,5,3320,759,16600,0,...,761,R,2025-01-24,1037.83,899.65,1111.87,954.31,865.07,912.63,5
96797,748,0,0,0,1,0,800,788,4000,0,...,748,R,2025-02-20,0.00,0.00,0.00,0.00,43.52,0.00,0


In [779]:
df_rams["current_account_nbr"] = df_rams["cu_account_nbr"]
df_rams.drop(['cu_account_nbr'], axis=1)

,cu_bhv_scr,ca_cash_bal_pct_crd_line,ca_cash_bal_pct_cash_line,cu_nbr_days_dlq,cu_nbr_of_plastics,ca_avg_utilz_lst_6_mnths,cu_cash_line_am,cu_crd_bureau_scr,cu_crd_line,cu_next_crd_line_rev_date,...,rb_crd_gr_new_crd_gr,cu_processing_date,mo_tot_sales_array_1,mo_tot_sales_array_2,mo_tot_sales_array_3,mo_tot_sales_array_4,mo_tot_sales_array_5,mo_tot_sales_array_6,ca_avg_utilz_lst_3_mnths,current_account_nbr
0,779,0,0,0,2,2,5000,782,25000,0,...,R,2024-12-17,997.56,521.07,647.86,1045.03,858.49,334.54,2,37eHiwRArQ0A7jhs
1,735,0,999,0,1,2,0,836,1500,0,...,R,2025-02-21,0.00,37.65,60.25,175.67,54.08,0.00,0,UV2Z1ODsmodPM2eO
2,745,0,0,0,1,2,8000,832,40000,0,...,R,2024-10-10,1024.55,578.80,665.81,816.43,2526.27,3129.23,1,VqweW0TxF93M2EF5
3,13,0,0,0,1,0,60,684,300,0,...,O,2025-01-27,0.00,0.00,0.00,0.00,0.00,0.00,0,pfKs8HDpmFV3b1zL
4,6,0,999,0,1,7,0,681,5200,0,...,P,2025-03-19,895.33,0.00,0.00,0.00,0.00,0.00,7,35VRb2UTGapQk6wE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96794,6,0,0,30,1,15,90,711,900,0,...,K,2024-11-14,0.00,185.10,0.00,0.00,0.00,0.00,15,Qc7bVxgbL7yLnwHk
96795,709,0,0,0,1,8,999999999999999,833,6700,0,...,R,2025-02-27,0.00,0.00,0.00,0.00,0.00,0.00,7,HaSj4cm2eeV1ddau
96796,755,0,0,0,1,5,3320,759,16600,0,...,R,2025-01-24,1037.83,899.65,1111.87,954.31,865.07,912.63,5,TUgAUpJzGvRCZPhr
96797,748,0,0,0,1,0,800,788,4000,0,...,R,2025-02-20,0.00,0.00,0.00,0.00,43.52,0.00,0,dCC3ETrdP8KNK5dM


In [780]:
df_fraud = pd.read_csv("/Users/kaitlinbaggot/Desktop/datathon 2025/fraud_claim_case_20250325.csv")
df_fraud

,current_account_nbr,case_id,reported_date,open_date,close_date,reopen_date,gross_fraud_amt,net_fraud_amt
0,ZB0SJEUntc0XyQuU,3202508301370,2025-03-23,2025-03-24,NaN,NaN,1789.82,1789.82
1,7zdxYCqGYTCFEKUb,4202235600151,2022-11-30,2022-12-22,2023-01-19,2023-01-10,71.87,0.00
2,VT29T3VPWCZ8KEkI,3202306200632,2023-03-02,2023-03-03,2023-04-14,2023-04-13,184.21,0.00
3,FqHTEAbjd4z65FWv,3202305801521,2023-02-26,2023-02-27,2023-04-09,2023-04-07,672.60,0.00
4,BG4qfLXo8chh5tIY,1202507200260,2025-03-11,2025-03-13,NaN,NaN,79.48,79.48
...,...,...,...,...,...,...,...,...
72,3Hj6EuxP2uay1tZR,3202315301546,2023-06-01,2023-06-02,2023-06-13,NaN,239.88,0.00
73,P586ALgLRL6ldsIW,1202208000284,2022-03-18,2022-03-21,2022-06-27,2022-06-24,142.36,0.00
74,apaOOlllTbXSAlVZ,1202507800267,2025-03-17,2025-03-19,2025-03-20,NaN,70.43,0.00
75,te0g3zNNAJ1cas4X,4202300200579,2023-01-01,2023-01-02,2023-01-19,NaN,25.00,0.00


In [781]:
df_world_trans = pd.read_csv("/Users/kaitlinbaggot/Desktop/datathon 2025/wrld_stor_tran_fact_20250325.csv")
df_world_trans

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt,product_amt,product_qty,...,first_purchase_ind,adj_orgn_tran_dt,curr_markup_fee,fcr_amount,fcr_flag,fcr_rate_of_exchange,frgn_curr_code,frgn_tran_amt,us_equiv_amt,posting_date
0,8DPC9336FwGf8TPz,SALE,253,2024-04-29,47.40,NaN,0,1,NaN,NaN,...,N,2024-03-17,0.0,NaN,NaN,NaN,840,47.40,0.0,2024-04-30
1,DRTiAtZURdSfMAB0,SALE,253,2024-06-25,87.13,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,87.13,0.0,2024-06-27
2,4y1CqjRDtP3w8Nb5,SALE,253,2024-08-06,30.00,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,30.00,0.0,2024-08-07
3,3dPDITyRPlzdq3wf,SALE,253,2024-09-05,32.30,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,32.30,0.0,2024-09-05
4,q1mnsZTeAaQH71AF,SALE,253,2024-12-30,4.40,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,4.40,0.0,2024-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053849,uqlll7MEoZkj3vur,SALE,253,2024-11-12,9.57,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,9.57,0.0,2024-11-13
1053850,ZGjYc8aETjtytUYR,SALE,253,2025-03-10,16.11,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,16.11,0.0,2025-03-11
1053851,4JpGONAzyfmoxi7A,SALE,253,2025-03-20,20.37,NaN,0,1,NaN,NaN,...,N,2025-02-26,0.0,NaN,NaN,NaN,840,20.37,0.0,2025-03-21
1053852,tUUUlJJZ2ETedT7g,SALE,253,2024-11-21,180.00,NaN,0,1,NaN,NaN,...,N,2024-11-12,0.0,NaN,NaN,NaN,840,180.00,0.0,2024-11-22


## Making Sense of Transactions Data

### Transactions Cleaning

In [782]:
df_transactions_highest = df_transactions.groupby("current_account_nbr")["transaction_amt"].max()
df_transactions_highest

current_account_nbr
00gwujJkd597VxLd      25.90
00iP5U82D8XwVQ9G     681.86
00oyr3QppAzjLws4     145.20
02WRVV8yusgcZWLW    5788.49
033o9yHYen3xoz6k    3182.10
                     ...   
zyikbceuTT3GcAH6       0.00
zz3nbtZXS41NZk0h      84.91
zzBy2qNM78aRV580     400.00
zzEuUBBmvGiVnabb    3635.15
zzR9PvG7dY9u5iHU     601.05
Name: transaction_amt, Length: 14099, dtype: float64

In [783]:
df_transactions_clean = df_transactions.drop(['product_amt', 'product_qty', 'invoice_nbr',
 'first_purchase_ind',
 'adj_orgn_tran_dt',
 'curr_markup_fee',
 'fcr_amount',
 'fcr_flag',
 'fcr_rate_of_exchange',
 'frgn_curr_code',
 'frgn_tran_amt',
 'us_equiv_amt',
 'posting_date'], axis=1)
df_transactions_clean

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
0,X7jfKh6xrPAB8Tx6,SALE,253,2024-06-05,15.78,NaN,0,1
1,yntD77AZDylS48Q4,SALE,253,2024-06-19,14.85,NaN,0,1
2,LIJPI0sK28Pa7fX2,SALE,253,2024-06-26,136.16,NaN,0,1
3,CMAr5Apxwdzpvoze,SALE,253,2024-08-15,8.74,NaN,0,1
4,eJSfTCGPvJulGzd3,SALE,253,2024-08-17,26.65,NaN,0,1
...,...,...,...,...,...,...,...,...
493331,ktpdvlbVRuywp2Po,SALE,253,2025-03-07,3.99,NaN,0,1
493332,xwgOt4lvAHr517sa,SALE,253,2024-07-27,6.13,NaN,0,1
493333,zp60CmOQNComnHDT,SALE,253,2025-02-04,15.62,NaN,0,1
493334,8M5fnshTsgW9mW8Y,PAYMENT,271,2025-02-05,100.00,NaN,0,0


In [784]:
df_transactions_clean = df_transactions_clean[df_transactions_clean["transaction_type"] != "ADJUSTMENT"]
df_transactions_clean

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
0,X7jfKh6xrPAB8Tx6,SALE,253,2024-06-05,15.78,NaN,0,1
1,yntD77AZDylS48Q4,SALE,253,2024-06-19,14.85,NaN,0,1
2,LIJPI0sK28Pa7fX2,SALE,253,2024-06-26,136.16,NaN,0,1
3,CMAr5Apxwdzpvoze,SALE,253,2024-08-15,8.74,NaN,0,1
4,eJSfTCGPvJulGzd3,SALE,253,2024-08-17,26.65,NaN,0,1
...,...,...,...,...,...,...,...,...
493331,ktpdvlbVRuywp2Po,SALE,253,2025-03-07,3.99,NaN,0,1
493332,xwgOt4lvAHr517sa,SALE,253,2024-07-27,6.13,NaN,0,1
493333,zp60CmOQNComnHDT,SALE,253,2025-02-04,15.62,NaN,0,1
493334,8M5fnshTsgW9mW8Y,PAYMENT,271,2025-02-05,100.00,NaN,0,0


In [785]:
df_transactions_clean = df_transactions_clean[df_transactions_clean["transaction_amt"] >= 0]
df_transactions_clean

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
0,X7jfKh6xrPAB8Tx6,SALE,253,2024-06-05,15.78,NaN,0,1
1,yntD77AZDylS48Q4,SALE,253,2024-06-19,14.85,NaN,0,1
2,LIJPI0sK28Pa7fX2,SALE,253,2024-06-26,136.16,NaN,0,1
3,CMAr5Apxwdzpvoze,SALE,253,2024-08-15,8.74,NaN,0,1
4,eJSfTCGPvJulGzd3,SALE,253,2024-08-17,26.65,NaN,0,1
...,...,...,...,...,...,...,...,...
493331,ktpdvlbVRuywp2Po,SALE,253,2025-03-07,3.99,NaN,0,1
493332,xwgOt4lvAHr517sa,SALE,253,2024-07-27,6.13,NaN,0,1
493333,zp60CmOQNComnHDT,SALE,253,2025-02-04,15.62,NaN,0,1
493334,8M5fnshTsgW9mW8Y,PAYMENT,271,2025-02-05,100.00,NaN,0,0


In [786]:
df_transactions_clean["transaction_date"].sort_values()

437454    2023-05-15
447830    2023-05-17
437065    2023-06-15
439981    2023-07-15
235708    2023-08-10
             ...    
322039    2025-03-24
462791    2025-03-24
377848    2025-03-24
316403    2025-03-24
466053    2025-03-24
Name: transaction_date, Length: 476099, dtype: object

In [787]:
df_transactions_Q1 = df_transactions_clean[df_transactions_clean["transaction_date"].astype(str).str.startswith(("2024-01", "2024-02", "2024-03"))]
df_transactions_Q1

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
11,ZSQKKuJzo3LNZetu,SALE,253,2024-03-27,36.34,NaN,0,1
27,QhYFOuQGrdYrJ6UP,SALE,253,2024-03-25,18.26,NaN,0,1
46,K1BMhCn54N0QzPWV,SALE,253,2024-03-29,310.43,NaN,0,1
154,0h3bLL0KB6KQ9yLi,PAYMENT,271,2024-03-29,125.00,NaN,0,0
176,JYcEYBYrKDNvSOCl,SALE,253,2024-03-26,13.07,NaN,0,1
...,...,...,...,...,...,...,...,...
492947,T6nbuIZLuMwVCIxs,SALE,253,2024-03-25,94.09,NaN,0,1
493024,KQg5vRR978TVG5RX,SALE,253,2024-03-30,99.63,NaN,0,1
493082,jI8tKgN5OZkw36AX,SALE,253,2024-03-27,48.47,NaN,0,1
493193,O3FJDKm7CuShNi1U,SALE,253,2024-03-27,53.61,NaN,0,1


In [788]:
df_transactions_Q2 = df_transactions_clean[df_transactions_clean["transaction_date"].astype(str).str.startswith(("2024-04", "2024-05", "2024-06"))]
df_transactions_Q2

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
0,X7jfKh6xrPAB8Tx6,SALE,253,2024-06-05,15.78,NaN,0,1
1,yntD77AZDylS48Q4,SALE,253,2024-06-19,14.85,NaN,0,1
2,LIJPI0sK28Pa7fX2,SALE,253,2024-06-26,136.16,NaN,0,1
12,skrJCHycIiIu0EtI,SALE,253,2024-06-17,5.00,NaN,0,1
17,GlGFcx3kCPAk9UfH,SALE,253,2024-04-06,33.21,NaN,0,1
...,...,...,...,...,...,...,...,...
493289,VAPiHkL69htyF011,SALE,253,2024-05-18,62.41,NaN,0,1
493292,ABoEIGMdfGiZwgf4,SALE,253,2024-04-17,200.73,NaN,0,1
493293,Dqdyi9rqaiQZcWOZ,SALE,253,2024-06-25,5.56,NaN,0,1
493306,8JVpw0YsqxQ1BsBK,SALE,253,2024-04-23,37.94,NaN,0,1


In [789]:
df_transactions_Q3 = df_transactions_clean[df_transactions_clean["transaction_date"].astype(str).str.startswith(("2024-07", "2024-08", "2024-09"))]
df_transactions_Q3

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
3,CMAr5Apxwdzpvoze,SALE,253,2024-08-15,8.74,NaN,0,1
4,eJSfTCGPvJulGzd3,SALE,253,2024-08-17,26.65,NaN,0,1
13,edqKl2I8yKGHvNu8,SALE,253,2024-08-16,107.74,NaN,0,1
29,adPZJ4iDtn41UyPp,SALE,253,2024-07-08,34.80,NaN,0,1
32,ySq4LWV2tp1Hdyw9,SALE,253,2024-07-18,21.79,NaN,0,1
...,...,...,...,...,...,...,...,...
493317,M1CjY8LQDAwCGFJn,SALE,253,2024-08-10,2.12,NaN,0,1
493319,NQ5OKPRlkpEt0lg7,PAYMENT,271,2024-09-05,15.00,NaN,0,0
493320,IKLMeHdyxDbz66CB,RETURN,255,2024-09-11,411.72,NaN,1,0
493322,h9qq3XKELV9RiA44,SALE,253,2024-09-14,87.97,NaN,0,1


In [790]:
df_transactions_Q4 = df_transactions_clean[df_transactions_clean["transaction_date"].astype(str).str.startswith(("2024-10", "2024-11", "2024-12"))]
df_transactions_Q4

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
5,AAGp1e2TujsQF0Tk,SALE,253,2024-10-20,25.68,NaN,0,1
6,p0X0C6QU59pYocnI,SALE,253,2024-12-14,358.62,NaN,0,1
14,Mb5wgeslW4eJ3Ctl,SALE,253,2024-10-28,163.63,NaN,0,1
15,RMZQcREtbIsbzi5u,SALE,253,2024-12-29,97.70,NaN,0,1
16,FePVFNPIklGA9tkE,SALE,253,2024-11-18,61.29,NaN,0,1
...,...,...,...,...,...,...,...,...
493323,ZPQQCuk31GGkGOSa,SALE,253,2024-10-08,2.16,NaN,0,1
493324,FPOLhsN1mFznZgBx,SALE,253,2024-10-12,15.29,NaN,0,1
493325,0hYf7EviiXnkuQOo,SALE,253,2024-12-03,8.60,NaN,0,1
493326,mKUxsiciAQAdUliE,PAYMENT,271,2024-10-22,10880.72,NaN,0,0


In [791]:
df_transactions_Q4["transaction_amt"].sort_values()

241585        0.00
305873        0.00
27857         0.00
325144        0.00
55833         0.00
            ...   
307163    20338.41
192830    23907.22
74211     25039.51
347348    31183.00
279323    40200.00
Name: transaction_amt, Length: 131032, dtype: float64

### World Transactions Cleaning

In [792]:
df_world_trans_clean = df_world_trans.drop(['product_amt', 'product_qty', 'invoice_nbr',
 'first_purchase_ind',
 'adj_orgn_tran_dt',
 'curr_markup_fee',
 'fcr_amount',
 'fcr_flag',
 'fcr_rate_of_exchange',
 'frgn_curr_code',
 'frgn_tran_amt',
 'us_equiv_amt',
 'posting_date'], axis=1)
df_world_trans_clean

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
0,8DPC9336FwGf8TPz,SALE,253,2024-04-29,47.40,NaN,0,1
1,DRTiAtZURdSfMAB0,SALE,253,2024-06-25,87.13,NaN,0,1
2,4y1CqjRDtP3w8Nb5,SALE,253,2024-08-06,30.00,NaN,0,1
3,3dPDITyRPlzdq3wf,SALE,253,2024-09-05,32.30,NaN,0,1
4,q1mnsZTeAaQH71AF,SALE,253,2024-12-30,4.40,NaN,0,1
...,...,...,...,...,...,...,...,...
1053849,uqlll7MEoZkj3vur,SALE,253,2024-11-12,9.57,NaN,0,1
1053850,ZGjYc8aETjtytUYR,SALE,253,2025-03-10,16.11,NaN,0,1
1053851,4JpGONAzyfmoxi7A,SALE,253,2025-03-20,20.37,NaN,0,1
1053852,tUUUlJJZ2ETedT7g,SALE,253,2024-11-21,180.00,NaN,0,1


In [793]:
df_world_trans_clean = df_world_trans_clean[df_world_trans_clean["transaction_type"] != "ADJUSTMENT"]
df_world_trans_clean

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
0,8DPC9336FwGf8TPz,SALE,253,2024-04-29,47.40,NaN,0,1
1,DRTiAtZURdSfMAB0,SALE,253,2024-06-25,87.13,NaN,0,1
2,4y1CqjRDtP3w8Nb5,SALE,253,2024-08-06,30.00,NaN,0,1
3,3dPDITyRPlzdq3wf,SALE,253,2024-09-05,32.30,NaN,0,1
4,q1mnsZTeAaQH71AF,SALE,253,2024-12-30,4.40,NaN,0,1
...,...,...,...,...,...,...,...,...
1053849,uqlll7MEoZkj3vur,SALE,253,2024-11-12,9.57,NaN,0,1
1053850,ZGjYc8aETjtytUYR,SALE,253,2025-03-10,16.11,NaN,0,1
1053851,4JpGONAzyfmoxi7A,SALE,253,2025-03-20,20.37,NaN,0,1
1053852,tUUUlJJZ2ETedT7g,SALE,253,2024-11-21,180.00,NaN,0,1


In [794]:
df_world_trans_clean = df_world_trans_clean[df_world_trans_clean["transaction_amt"] >= 0]
df_world_trans_clean

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
0,8DPC9336FwGf8TPz,SALE,253,2024-04-29,47.40,NaN,0,1
1,DRTiAtZURdSfMAB0,SALE,253,2024-06-25,87.13,NaN,0,1
2,4y1CqjRDtP3w8Nb5,SALE,253,2024-08-06,30.00,NaN,0,1
3,3dPDITyRPlzdq3wf,SALE,253,2024-09-05,32.30,NaN,0,1
4,q1mnsZTeAaQH71AF,SALE,253,2024-12-30,4.40,NaN,0,1
...,...,...,...,...,...,...,...,...
1053849,uqlll7MEoZkj3vur,SALE,253,2024-11-12,9.57,NaN,0,1
1053850,ZGjYc8aETjtytUYR,SALE,253,2025-03-10,16.11,NaN,0,1
1053851,4JpGONAzyfmoxi7A,SALE,253,2025-03-20,20.37,NaN,0,1
1053852,tUUUlJJZ2ETedT7g,SALE,253,2024-11-21,180.00,NaN,0,1


In [795]:
df_world_trans_Q1 = df_world_trans_clean[df_world_trans_clean["transaction_date"].astype(str).str.startswith(("2024-01", "2024-02", "2024-03"))]
df_world_trans_Q1

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
68,QaNKjwqCx045rzLr,SALE,253,2024-03-29,18.66,NaN,0,1
117,NJ5mp4zsRzpEShey,SALE,253,2024-03-30,44.92,NaN,0,1
118,XZIpMKbx7PZIOeOy,SALE,253,2024-03-28,99.70,NaN,0,1
167,csRVJNQ9lek65m2z,SALE,253,2024-03-26,25.28,NaN,0,1
168,wiYwqGm4ht2naNYH,SALE,253,2024-03-30,5.74,NaN,0,1
...,...,...,...,...,...,...,...,...
1053651,3RiM5svLt5UoMaTT,RETURN,255,2024-03-29,10.60,NaN,1,0
1053710,7PfwMQJdajj1tlpn,SALE,253,2024-03-28,243.88,NaN,0,1
1053769,9RUzMCcLCMAUvx6n,SALE,253,2024-03-29,8.40,NaN,0,1
1053785,7Dzj9nR8CI2a2DtC,SALE,253,2024-03-31,5.00,NaN,0,1


In [796]:
df_world_trans_Q2 = df_world_trans_clean[df_world_trans_clean["transaction_date"].astype(str).str.startswith(("2024-04", "2024-05", "2024-06"))]
df_world_trans_Q2

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
0,8DPC9336FwGf8TPz,SALE,253,2024-04-29,47.40,NaN,0,1
1,DRTiAtZURdSfMAB0,SALE,253,2024-06-25,87.13,NaN,0,1
10,gIoHHQ3laMHoedVO,SALE,253,2024-04-03,9.06,NaN,0,1
11,0LfFC0f2GAC0aS1f,SALE,253,2024-04-22,278.00,NaN,0,1
12,QmvwvXtqHJm8Hx7b,SALE,253,2024-05-03,3.00,NaN,0,1
...,...,...,...,...,...,...,...,...
1053811,Tbeo412vddxmO7FZ,SALE,253,2024-04-22,281.33,NaN,0,1
1053816,LjB2nrI82RbjXXrt,SALE,253,2024-05-05,4.99,NaN,0,1
1053818,Xm3AmRGUAg62BMpb,SALE,253,2024-05-30,24.01,NaN,0,1
1053819,4ILzV5Ka6nh4lWsB,SALE,253,2024-05-05,13.66,NaN,0,1


In [797]:
df_world_trans_Q3 = df_world_trans_clean[df_world_trans_clean["transaction_date"].astype(str).str.startswith(("2024-07", "2024-08", "2024-09"))]
df_world_trans_Q3

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
2,4y1CqjRDtP3w8Nb5,SALE,253,2024-08-06,30.00,NaN,0,1
3,3dPDITyRPlzdq3wf,SALE,253,2024-09-05,32.30,NaN,0,1
13,lP4GKw5e42TtVjyi,SALE,253,2024-07-23,328.10,NaN,0,1
26,6CdqBsCaDEsevnsp,SALE,253,2024-07-15,28.03,NaN,0,1
31,qVsKGsk8ahYae4kX,SALE,253,2024-07-01,4.19,NaN,0,1
...,...,...,...,...,...,...,...,...
1053817,OXSY4vvAXSrrmV1V,SALE,253,2024-09-03,76.10,NaN,0,1
1053820,lP4GKw5e42TtVjyi,SALE,253,2024-07-23,328.10,NaN,0,1
1053823,pOKwgCrIXtjUkdZr,SALE,253,2024-08-15,16.66,NaN,0,1
1053824,J5KM6VrXlpeiISLM,SALE,253,2024-09-21,30.66,NaN,0,1


In [798]:
df_world_trans_Q4 = df_world_trans_clean[df_world_trans_clean["transaction_date"].astype(str).str.startswith(("2024-10", "2024-11", "2024-12"))]
df_world_trans_Q4

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
4,q1mnsZTeAaQH71AF,SALE,253,2024-12-30,4.40,NaN,0,1
5,m7ca9TUV8qbWJkcb,SALE,253,2024-12-29,22.72,NaN,0,1
9,cmzM4aYbijSJMEG2,SALE,253,2024-10-08,12.00,NaN,0,1
14,pOKwgCrIXtjUkdZr,SALE,253,2024-12-04,27.66,NaN,0,1
15,S9R9Edu0bMIyfUnE,SALE,253,2024-12-19,59.37,NaN,0,1
...,...,...,...,...,...,...,...,...
1053844,9gJsz8tSaVOwokPd,SALE,253,2024-10-12,25.67,NaN,0,1
1053847,WBBqIOirecnTtAoI,SALE,253,2024-10-18,17.48,NaN,0,1
1053849,uqlll7MEoZkj3vur,SALE,253,2024-11-12,9.57,NaN,0,1
1053852,tUUUlJJZ2ETedT7g,SALE,253,2024-11-21,180.00,NaN,0,1


## Getting Q1-Q3 Trends

### Transactions Q1 - Q3

In [799]:
df_transactions_Q3_sum = df_transactions_Q3.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_transactions_Q3_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
00iP5U82D8XwVQ9G,2262.72,14.692987
00oyr3QppAzjLws4,390.04,78.008000
033o9yHYen3xoz6k,5386.87,414.374615
034bM166vNmgLiIA,1107.85,369.283333
03n28YA8ljfM9tor,622.16,311.080000
...,...,...
zxn3AYXktnnqqYRJ,1628.11,116.293571
zxwuHFEBf4ERmY9F,1022.80,73.057143
zyZhjzJwhpOgSvmc,1909.55,146.888462


In [800]:
df_transactions_Q2_sum = df_transactions_Q2.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_transactions_Q2_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
00iP5U82D8XwVQ9G,1301.59,14.303187
00oyr3QppAzjLws4,625.52,78.190000
033o9yHYen3xoz6k,4911.03,306.939375
034bM166vNmgLiIA,19.98,9.990000
03n28YA8ljfM9tor,111.11,111.110000
...,...,...
zxn3AYXktnnqqYRJ,5349.70,297.205556
zxwuHFEBf4ERmY9F,8787.41,351.496400
zyZhjzJwhpOgSvmc,495.26,99.052000


In [801]:
df_transactions_Q1_sum = df_transactions_Q1.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_transactions_Q1_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
00iP5U82D8XwVQ9G,93.23,6.659286
033o9yHYen3xoz6k,1252.33,1252.330000
03tBhY3OXJglD820,150.00,150.000000
05AfSsd6epxXiK01,136.81,34.202500
05g3It0iVfAnv3ur,5.42,5.420000
...,...,...
zsYIael1LtCzyZNe,151.08,21.582857
zulSUhlkOQHtZWvu,126.99,63.495000
zxn3AYXktnnqqYRJ,0.00,0.000000


In [802]:
df_transactions_Q1_to_Q3 = pd.merge(df_transactions_Q1_sum, df_transactions_Q2_sum, on="current_account_nbr")
df_transactions_Q1_to_Q3

,total_spent_x,avg_transaction_amt_x,total_spent_y,avg_transaction_amt_y
current_account_nbr,,,,
00iP5U82D8XwVQ9G,93.23,6.659286,1301.59,14.303187
033o9yHYen3xoz6k,1252.33,1252.330000,4911.03,306.939375
03tBhY3OXJglD820,150.00,150.000000,2150.00,358.333333
05AfSsd6epxXiK01,136.81,34.202500,16891.64,383.900909
05g3It0iVfAnv3ur,5.42,5.420000,258.09,17.206000
...,...,...,...,...
zsYIael1LtCzyZNe,151.08,21.582857,5480.26,219.210400
zulSUhlkOQHtZWvu,126.99,63.495000,1706.10,60.932143
zxn3AYXktnnqqYRJ,0.00,0.000000,5349.70,297.205556


In [803]:
df_transactions_Q1_to_Q3 = pd.merge(df_transactions_Q1_to_Q3, df_transactions_Q3_sum, on="current_account_nbr")
df_transactions_Q1_to_Q3

,total_spent_x,avg_transaction_amt_x,total_spent_y,avg_transaction_amt_y,total_spent,avg_transaction_amt
current_account_nbr,,,,,,
00iP5U82D8XwVQ9G,93.23,6.659286,1301.59,14.303187,2262.72,14.692987
033o9yHYen3xoz6k,1252.33,1252.330000,4911.03,306.939375,5386.87,414.374615
03tBhY3OXJglD820,150.00,150.000000,2150.00,358.333333,471.00,117.750000
05AfSsd6epxXiK01,136.81,34.202500,16891.64,383.900909,18025.49,383.521064
05g3It0iVfAnv3ur,5.42,5.420000,258.09,17.206000,1391.01,51.518889
...,...,...,...,...,...,...
zsYIael1LtCzyZNe,151.08,21.582857,5480.26,219.210400,3435.57,180.819474
zulSUhlkOQHtZWvu,126.99,63.495000,1706.10,60.932143,1529.97,72.855714
zxn3AYXktnnqqYRJ,0.00,0.000000,5349.70,297.205556,1628.11,116.293571


In [804]:
df_transactions_Q1_to_Q3 = df_transactions_Q1_to_Q3.rename(columns={'total_spent_x': 'total_Q1', 
                                                                    'avg_transaction_amt_x': 'avg_trans_Q1', 
                                                                    'total_spent_y': 'total_Q2', 
                                                                    'avg_transaction_amt_y': 'avg_trans_Q2',
                                                                    'total_spent': 'total_Q3', 
                                                                    'avg_transaction_amt': 'avg_trans_Q3'})
df_transactions_Q1_to_Q3

,total_Q1,avg_trans_Q1,total_Q2,avg_trans_Q2,total_Q3,avg_trans_Q3
current_account_nbr,,,,,,
00iP5U82D8XwVQ9G,93.23,6.659286,1301.59,14.303187,2262.72,14.692987
033o9yHYen3xoz6k,1252.33,1252.330000,4911.03,306.939375,5386.87,414.374615
03tBhY3OXJglD820,150.00,150.000000,2150.00,358.333333,471.00,117.750000
05AfSsd6epxXiK01,136.81,34.202500,16891.64,383.900909,18025.49,383.521064
05g3It0iVfAnv3ur,5.42,5.420000,258.09,17.206000,1391.01,51.518889
...,...,...,...,...,...,...
zsYIael1LtCzyZNe,151.08,21.582857,5480.26,219.210400,3435.57,180.819474
zulSUhlkOQHtZWvu,126.99,63.495000,1706.10,60.932143,1529.97,72.855714
zxn3AYXktnnqqYRJ,0.00,0.000000,5349.70,297.205556,1628.11,116.293571


### World Transactions Q1 - Q3

In [805]:
df_world_trans_Q3_sum = df_world_trans_Q3.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_world_trans_Q3_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
033o9yHYen3xoz6k,4811.56,48.115600
034bM166vNmgLiIA,1507.72,55.841481
049sX01uVFhaa0i2,7733.02,48.031180
05AfSsd6epxXiK01,20164.39,110.187923
05oLsFWaSLYzc5hv,1483.76,17.663810
...,...,...
zuPY0iifqZT124F8,210.00,70.000000
zwtnyO2BGLGWUf7r,728.03,60.669167
zyZhjzJwhpOgSvmc,1405.01,36.025897


In [806]:
df_world_trans_Q2_sum = df_world_trans_Q2.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_world_trans_Q2_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
033o9yHYen3xoz6k,3944.67,41.522842
049sX01uVFhaa0i2,5679.60,40.568571
05AfSsd6epxXiK01,12531.70,62.658500
05oLsFWaSLYzc5hv,2300.34,23.003400
062wBe4JtO4ij7Mp,11756.97,67.182686
...,...,...
zuPY0iifqZT124F8,210.00,70.000000
zwtnyO2BGLGWUf7r,1297.21,86.480667
zyZhjzJwhpOgSvmc,615.72,36.218824


In [807]:
df_world_trans_Q1_sum = df_world_trans_Q1.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_world_trans_Q1_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
033o9yHYen3xoz6k,128.54,32.135000
049sX01uVFhaa0i2,203.79,29.112857
05AfSsd6epxXiK01,716.21,29.842083
05oLsFWaSLYzc5hv,73.70,12.283333
062wBe4JtO4ij7Mp,1135.44,70.965000
...,...,...
zp60CmOQNComnHDT,56.91,18.970000
zqLSCw6bJGUB019O,281.80,281.800000
zsYIael1LtCzyZNe,296.44,37.055000


In [808]:
df_world_trans_Q1_to_Q3 = pd.merge(df_world_trans_Q1_sum, df_world_trans_Q2_sum, on="current_account_nbr")
df_world_trans_Q1_to_Q3

,total_spent_x,avg_transaction_amt_x,total_spent_y,avg_transaction_amt_y
current_account_nbr,,,,
033o9yHYen3xoz6k,128.54,32.135000,3944.67,41.522842
049sX01uVFhaa0i2,203.79,29.112857,5679.60,40.568571
05AfSsd6epxXiK01,716.21,29.842083,12531.70,62.658500
05oLsFWaSLYzc5hv,73.70,12.283333,2300.34,23.003400
062wBe4JtO4ij7Mp,1135.44,70.965000,11756.97,67.182686
...,...,...,...,...
zoD9wnDwZFJyCtcj,66.59,13.318000,1866.99,20.744333
zp60CmOQNComnHDT,56.91,18.970000,7328.97,43.366686
zsYIael1LtCzyZNe,296.44,37.055000,3035.60,151.780000


In [809]:
df_world_trans_Q1_to_Q3 = pd.merge(df_world_trans_Q1_to_Q3, df_world_trans_Q3_sum, on="current_account_nbr")
df_world_trans_Q1_to_Q3

,total_spent_x,avg_transaction_amt_x,total_spent_y,avg_transaction_amt_y,total_spent,avg_transaction_amt
current_account_nbr,,,,,,
033o9yHYen3xoz6k,128.54,32.135000,3944.67,41.522842,4811.56,48.115600
049sX01uVFhaa0i2,203.79,29.112857,5679.60,40.568571,7733.02,48.031180
05AfSsd6epxXiK01,716.21,29.842083,12531.70,62.658500,20164.39,110.187923
05oLsFWaSLYzc5hv,73.70,12.283333,2300.34,23.003400,1483.76,17.663810
062wBe4JtO4ij7Mp,1135.44,70.965000,11756.97,67.182686,12397.09,85.497172
...,...,...,...,...,...,...
zoD9wnDwZFJyCtcj,66.59,13.318000,1866.99,20.744333,1644.25,16.279703
zp60CmOQNComnHDT,56.91,18.970000,7328.97,43.366686,21241.50,136.163462
zsYIael1LtCzyZNe,296.44,37.055000,3035.60,151.780000,1082.72,56.985263


In [810]:
df_world_trans_Q1_to_Q3 = df_world_trans_Q1_to_Q3.rename(columns={'total_spent_x': 'total_Q1', 
                                                                    'avg_transaction_amt_x': 'avg_trans_Q1', 
                                                                    'total_spent_y': 'total_Q2', 
                                                                    'avg_transaction_amt_y': 'avg_trans_Q2',
                                                                    'total_spent': 'total_Q3', 
                                                                    'avg_transaction_amt': 'avg_trans_Q3'})
df_world_trans_Q1_to_Q3

,total_Q1,avg_trans_Q1,total_Q2,avg_trans_Q2,total_Q3,avg_trans_Q3
current_account_nbr,,,,,,
033o9yHYen3xoz6k,128.54,32.135000,3944.67,41.522842,4811.56,48.115600
049sX01uVFhaa0i2,203.79,29.112857,5679.60,40.568571,7733.02,48.031180
05AfSsd6epxXiK01,716.21,29.842083,12531.70,62.658500,20164.39,110.187923
05oLsFWaSLYzc5hv,73.70,12.283333,2300.34,23.003400,1483.76,17.663810
062wBe4JtO4ij7Mp,1135.44,70.965000,11756.97,67.182686,12397.09,85.497172
...,...,...,...,...,...,...
zoD9wnDwZFJyCtcj,66.59,13.318000,1866.99,20.744333,1644.25,16.279703
zp60CmOQNComnHDT,56.91,18.970000,7328.97,43.366686,21241.50,136.163462
zsYIael1LtCzyZNe,296.44,37.055000,3035.60,151.780000,1082.72,56.985263


### Combining Non-World & World Transactions Q1 - Q3 

In [811]:
df_total_trans_Q1_to_Q3 = pd.merge(df_transactions_Q1_to_Q3, df_world_trans_Q1_to_Q3, on="current_account_nbr")
df_total_trans_Q1_to_Q3

,total_Q1_x,avg_trans_Q1_x,total_Q2_x,avg_trans_Q2_x,total_Q3_x,avg_trans_Q3_x,total_Q1_y,avg_trans_Q1_y,total_Q2_y,avg_trans_Q2_y,total_Q3_y,avg_trans_Q3_y
current_account_nbr,,,,,,,,,,,,
033o9yHYen3xoz6k,1252.33,1252.330000,4911.03,306.939375,5386.87,414.374615,128.54,32.135000,3944.67,41.522842,4811.56,48.115600
05AfSsd6epxXiK01,136.81,34.202500,16891.64,383.900909,18025.49,383.521064,716.21,29.842083,12531.70,62.658500,20164.39,110.187923
05oLsFWaSLYzc5hv,135.66,45.220000,3437.04,214.815000,2447.56,106.415652,73.70,12.283333,2300.34,23.003400,1483.76,17.663810
062wBe4JtO4ij7Mp,182.49,60.830000,15277.06,412.893514,17183.96,536.998750,1135.44,70.965000,11756.97,67.182686,12397.09,85.497172
06SijOFpCXUvw9hs,4901.97,2450.985000,13331.87,555.494583,5988.97,239.558800,637.47,49.036154,7853.50,153.990196,3883.71,45.690706
...,...,...,...,...,...,...,...,...,...,...,...,...
znzcg2AkVtchRVjt,70.71,35.355000,5536.92,190.928276,5485.21,203.155926,274.79,91.596667,2459.49,57.197442,2546.90,70.747222
zoBLVNjS9F2gDeYk,114.57,57.285000,21313.72,507.469524,39898.41,1049.958158,698.49,29.103750,29980.20,97.974510,20571.44,56.514945
zoD9wnDwZFJyCtcj,651.92,651.920000,2525.99,132.946842,3514.28,167.346667,66.59,13.318000,1866.99,20.744333,1644.25,16.279703


In [812]:
df_total_trans_Q1_to_Q3["total_Q1"] = df_total_trans_Q1_to_Q3["total_Q1_x"] + df_total_trans_Q1_to_Q3["total_Q1_y"]
df_total_trans_Q1_to_Q3["total_Q2"] = df_total_trans_Q1_to_Q3["total_Q2_x"] + df_total_trans_Q1_to_Q3["total_Q2_y"]
df_total_trans_Q1_to_Q3["total_Q3"] = df_total_trans_Q1_to_Q3["total_Q3_x"] + df_total_trans_Q1_to_Q3["total_Q3_y"]
df_total_trans_Q1_to_Q3

,total_Q1_x,avg_trans_Q1_x,total_Q2_x,avg_trans_Q2_x,total_Q3_x,avg_trans_Q3_x,total_Q1_y,avg_trans_Q1_y,total_Q2_y,avg_trans_Q2_y,total_Q3_y,avg_trans_Q3_y,total_Q1,total_Q2,total_Q3
current_account_nbr,,,,,,,,,,,,,,,
033o9yHYen3xoz6k,1252.33,1252.330000,4911.03,306.939375,5386.87,414.374615,128.54,32.135000,3944.67,41.522842,4811.56,48.115600,1380.87,8855.70,10198.43
05AfSsd6epxXiK01,136.81,34.202500,16891.64,383.900909,18025.49,383.521064,716.21,29.842083,12531.70,62.658500,20164.39,110.187923,853.02,29423.34,38189.88
05oLsFWaSLYzc5hv,135.66,45.220000,3437.04,214.815000,2447.56,106.415652,73.70,12.283333,2300.34,23.003400,1483.76,17.663810,209.36,5737.38,3931.32
062wBe4JtO4ij7Mp,182.49,60.830000,15277.06,412.893514,17183.96,536.998750,1135.44,70.965000,11756.97,67.182686,12397.09,85.497172,1317.93,27034.03,29581.05
06SijOFpCXUvw9hs,4901.97,2450.985000,13331.87,555.494583,5988.97,239.558800,637.47,49.036154,7853.50,153.990196,3883.71,45.690706,5539.44,21185.37,9872.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
znzcg2AkVtchRVjt,70.71,35.355000,5536.92,190.928276,5485.21,203.155926,274.79,91.596667,2459.49,57.197442,2546.90,70.747222,345.50,7996.41,8032.11
zoBLVNjS9F2gDeYk,114.57,57.285000,21313.72,507.469524,39898.41,1049.958158,698.49,29.103750,29980.20,97.974510,20571.44,56.514945,813.06,51293.92,60469.85
zoD9wnDwZFJyCtcj,651.92,651.920000,2525.99,132.946842,3514.28,167.346667,66.59,13.318000,1866.99,20.744333,1644.25,16.279703,718.51,4392.98,5158.53


In [813]:
df_total_trans_Q1_to_Q3 = df_total_trans_Q1_to_Q3.drop(['total_Q1_x', 'total_Q1_y', 'total_Q2_x', 'total_Q2_y', 'total_Q3_x', 'total_Q3_y'], axis=1)
df_total_trans_Q1_to_Q3

,avg_trans_Q1_x,avg_trans_Q2_x,avg_trans_Q3_x,avg_trans_Q1_y,avg_trans_Q2_y,avg_trans_Q3_y,total_Q1,total_Q2,total_Q3
current_account_nbr,,,,,,,,,
033o9yHYen3xoz6k,1252.330000,306.939375,414.374615,32.135000,41.522842,48.115600,1380.87,8855.70,10198.43
05AfSsd6epxXiK01,34.202500,383.900909,383.521064,29.842083,62.658500,110.187923,853.02,29423.34,38189.88
05oLsFWaSLYzc5hv,45.220000,214.815000,106.415652,12.283333,23.003400,17.663810,209.36,5737.38,3931.32
062wBe4JtO4ij7Mp,60.830000,412.893514,536.998750,70.965000,67.182686,85.497172,1317.93,27034.03,29581.05
06SijOFpCXUvw9hs,2450.985000,555.494583,239.558800,49.036154,153.990196,45.690706,5539.44,21185.37,9872.68
...,...,...,...,...,...,...,...,...,...
znzcg2AkVtchRVjt,35.355000,190.928276,203.155926,91.596667,57.197442,70.747222,345.50,7996.41,8032.11
zoBLVNjS9F2gDeYk,57.285000,507.469524,1049.958158,29.103750,97.974510,56.514945,813.06,51293.92,60469.85
zoD9wnDwZFJyCtcj,651.920000,132.946842,167.346667,13.318000,20.744333,16.279703,718.51,4392.98,5158.53


## Getting Q4 Total Spending

In [814]:
df_transactions_Q4_sum = df_transactions_Q4.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_transactions_Q4_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
00iP5U82D8XwVQ9G,3449.74,13.217395
00oyr3QppAzjLws4,515.96,73.708571
033o9yHYen3xoz6k,8042.30,473.076471
034bM166vNmgLiIA,488.01,162.670000
03cqv0gFjEIiQG0x,40.00,40.000000
...,...,...
zxwuHFEBf4ERmY9F,1899.68,61.280000
zyZhjzJwhpOgSvmc,2304.89,192.074167
zz3nbtZXS41NZk0h,239.82,47.964000


In [815]:
df_world_trans_Q4_sum = df_world_trans_Q4.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_world_trans_Q4_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
033o9yHYen3xoz6k,6529.56,46.639714
034bM166vNmgLiIA,719.39,37.862632
049sX01uVFhaa0i2,5720.27,45.398968
05AfSsd6epxXiK01,16524.12,76.500556
05oLsFWaSLYzc5hv,873.37,58.224667
...,...,...
zuPY0iifqZT124F8,210.00,70.000000
zwtnyO2BGLGWUf7r,2533.08,70.363333
zxOqFgWke0qPQZIO,0.00,0.000000


In [816]:
df_trans_Q4_merged = pd.merge(df_transactions_Q4_sum, df_world_trans_Q4_sum, on="current_account_nbr")
df_trans_Q4_merged

,total_spent_x,avg_transaction_amt_x,total_spent_y,avg_transaction_amt_y
current_account_nbr,,,,
033o9yHYen3xoz6k,8042.30,473.076471,6529.56,46.639714
034bM166vNmgLiIA,488.01,162.670000,719.39,37.862632
049sX01uVFhaa0i2,7983.33,798.333000,5720.27,45.398968
05AfSsd6epxXiK01,22205.66,370.094333,16524.12,76.500556
05oLsFWaSLYzc5hv,2240.80,112.040000,873.37,58.224667
...,...,...,...,...
zsYIael1LtCzyZNe,4557.63,162.772500,1565.53,44.729429
zwtnyO2BGLGWUf7r,700.00,233.333333,2533.08,70.363333
zxOqFgWke0qPQZIO,259.88,64.970000,0.00,0.000000


In [817]:
df_trans_Q4_merged["last_Q4_spending"] = df_trans_Q4_merged["total_spent_x"] + df_trans_Q4_merged["total_spent_y"]
df_trans_Q4_merged

,total_spent_x,avg_transaction_amt_x,total_spent_y,avg_transaction_amt_y,last_Q4_spending
current_account_nbr,,,,,
033o9yHYen3xoz6k,8042.30,473.076471,6529.56,46.639714,14571.86
034bM166vNmgLiIA,488.01,162.670000,719.39,37.862632,1207.40
049sX01uVFhaa0i2,7983.33,798.333000,5720.27,45.398968,13703.60
05AfSsd6epxXiK01,22205.66,370.094333,16524.12,76.500556,38729.78
05oLsFWaSLYzc5hv,2240.80,112.040000,873.37,58.224667,3114.17
...,...,...,...,...,...
zsYIael1LtCzyZNe,4557.63,162.772500,1565.53,44.729429,6123.16
zwtnyO2BGLGWUf7r,700.00,233.333333,2533.08,70.363333,3233.08
zxOqFgWke0qPQZIO,259.88,64.970000,0.00,0.000000,259.88


In [818]:
df_trans_Q4_merged["avg_transaction_amt"] = df_trans_Q4_merged["avg_transaction_amt_x"]
df_trans_Q4_merged["avg_world_transaction_amt"] = df_trans_Q4_merged["avg_transaction_amt_y"]
df_trans_Q4_merged

,total_spent_x,avg_transaction_amt_x,total_spent_y,avg_transaction_amt_y,last_Q4_spending,avg_transaction_amt,avg_world_transaction_amt
current_account_nbr,,,,,,,
033o9yHYen3xoz6k,8042.30,473.076471,6529.56,46.639714,14571.86,473.076471,46.639714
034bM166vNmgLiIA,488.01,162.670000,719.39,37.862632,1207.40,162.670000,37.862632
049sX01uVFhaa0i2,7983.33,798.333000,5720.27,45.398968,13703.60,798.333000,45.398968
05AfSsd6epxXiK01,22205.66,370.094333,16524.12,76.500556,38729.78,370.094333,76.500556
05oLsFWaSLYzc5hv,2240.80,112.040000,873.37,58.224667,3114.17,112.040000,58.224667
...,...,...,...,...,...,...,...
zsYIael1LtCzyZNe,4557.63,162.772500,1565.53,44.729429,6123.16,162.772500,44.729429
zwtnyO2BGLGWUf7r,700.00,233.333333,2533.08,70.363333,3233.08,233.333333,70.363333
zxOqFgWke0qPQZIO,259.88,64.970000,0.00,0.000000,259.88,64.970000,0.000000


In [819]:
df_trans_Q4_merged = df_trans_Q4_merged.drop(['avg_transaction_amt_x', 'avg_transaction_amt_y', 'total_spent_x', 'total_spent_y'], axis=1)
df_trans_Q4_merged

,last_Q4_spending,avg_transaction_amt,avg_world_transaction_amt
current_account_nbr,,,
033o9yHYen3xoz6k,14571.86,473.076471,46.639714
034bM166vNmgLiIA,1207.40,162.670000,37.862632
049sX01uVFhaa0i2,13703.60,798.333000,45.398968
05AfSsd6epxXiK01,38729.78,370.094333,76.500556
05oLsFWaSLYzc5hv,3114.17,112.040000,58.224667
...,...,...,...
zsYIael1LtCzyZNe,6123.16,162.772500,44.729429
zwtnyO2BGLGWUf7r,3233.08,233.333333,70.363333
zxOqFgWke0qPQZIO,259.88,64.970000,0.000000


## Combining with Account Features Data

Needed columns: cu_card_line, cu_bhv_scr, cu_nbr_days_dlq, ca_avg_utilz_lst_6_mnths, ca_nsf_count_lst_12_months, cu_otb, rb_new_bhv_scr, ca_avg_utilz_lst_3_mnths

In [820]:
col_keep = ['cu_crd_line', 'current_account_nbr', 'cu_bhv_scr', 'cu_nbr_days_dlq', 'ca_avg_utilz_lst_6_mnths', 'ca_nsf_count_lst_12_months', 'cu_otb', 'rb_new_bhv_scr', 'ca_avg_utilz_lst_3_mnths', 'ca_max_dlq_lst_6_mnths', 'cu_processing_date']

In [821]:
df_rams_clean = df_rams[col_keep]
df_rams_clean

,cu_crd_line,current_account_nbr,cu_bhv_scr,cu_nbr_days_dlq,ca_avg_utilz_lst_6_mnths,ca_nsf_count_lst_12_months,cu_otb,rb_new_bhv_scr,ca_avg_utilz_lst_3_mnths,ca_max_dlq_lst_6_mnths,cu_processing_date
0,25000,37eHiwRArQ0A7jhs,779,0,2,0,24002.44,776,2,0,2024-12-17
1,1500,UV2Z1ODsmodPM2eO,735,0,2,0,1500.00,735,0,0,2025-02-21
2,40000,VqweW0TxF93M2EF5,745,0,2,0,38982.45,748,1,0,2024-10-10
3,300,pfKs8HDpmFV3b1zL,13,0,0,0,300.00,13,0,0,2025-01-27
4,5200,35VRb2UTGapQk6wE,6,0,7,0,4368.27,6,7,0,2025-03-19
...,...,...,...,...,...,...,...,...,...,...,...
96794,900,Qc7bVxgbL7yLnwHk,6,30,15,0,679.76,628,15,1,2024-11-14
96795,6700,HaSj4cm2eeV1ddau,709,0,8,0,6230.00,703,7,0,2025-02-27
96796,16600,TUgAUpJzGvRCZPhr,755,0,5,0,15562.17,761,5,0,2025-01-24
96797,4000,dCC3ETrdP8KNK5dM,748,0,0,0,4000.00,748,0,0,2025-02-20


In [822]:
df_rams_sorted = df_rams_clean.sort_values(by='current_account_nbr')
df_rams_sorted

,cu_crd_line,current_account_nbr,cu_bhv_scr,cu_nbr_days_dlq,ca_avg_utilz_lst_6_mnths,ca_nsf_count_lst_12_months,cu_otb,rb_new_bhv_scr,ca_avg_utilz_lst_3_mnths,ca_max_dlq_lst_6_mnths,cu_processing_date
36973,3500,00iP5U82D8XwVQ9G,742,0,11,0,2980.80,744,11,0,2024-10-25
6623,3500,00iP5U82D8XwVQ9G,735,0,14,0,3005.81,735,17,0,2025-01-26
91888,3500,00iP5U82D8XwVQ9G,735,0,14,0,3165.90,742,14,0,2025-02-23
13493,3500,00iP5U82D8XwVQ9G,744,0,12,0,2863.99,738,15,0,2024-11-25
31420,3500,00iP5U82D8XwVQ9G,738,0,12,0,2818.14,735,17,0,2024-12-26
...,...,...,...,...,...,...,...,...,...,...,...
51612,9900,zzztTVczEiGgAEJi,13,0,0,0,9900.00,13,0,0,2024-11-27
15565,9900,zzztTVczEiGgAEJi,13,0,0,0,9900.00,13,0,0,2024-10-28
13117,9900,zzztTVczEiGgAEJi,13,0,0,0,9900.00,13,0,0,2025-01-28
25254,9900,zzztTVczEiGgAEJi,13,0,0,0,9900.00,13,0,0,2024-12-27


In [823]:
df_rams_sorted['cu_processing_date'] = pd.to_datetime(df_rams_sorted['cu_processing_date'])

In [824]:
df_rams_recent = df_rams_sorted.loc[df_rams_sorted.groupby('current_account_nbr')['cu_processing_date'].idxmax()]
df_rams_recent

,cu_crd_line,current_account_nbr,cu_bhv_scr,cu_nbr_days_dlq,ca_avg_utilz_lst_6_mnths,ca_nsf_count_lst_12_months,cu_otb,rb_new_bhv_scr,ca_avg_utilz_lst_3_mnths,ca_max_dlq_lst_6_mnths,cu_processing_date
91888,3500,00iP5U82D8XwVQ9G,735,0,14,0,3165.90,742,14,0,2025-02-23
78527,2900,00oyr3QppAzjLws4,661,0,90,0,367.58,679,90,0,2025-03-10
18577,1750,01SEhQXHbPJRc1Go,12,0,0,0,1750.00,12,0,0,2025-02-27
40876,400,01wbqz3obYPYxLvR,11,0,0,0,400.00,11,0,0,2025-03-13
32410,2000,020qpQKv0isIguXC,12,0,0,0,2000.00,12,0,0,2025-03-23
...,...,...,...,...,...,...,...,...,...,...,...
28670,14800,zzBy2qNM78aRV580,749,0,1,0,14800.00,746,0,0,2025-03-23
37689,6000,zzEuUBBmvGiVnabb,724,0,15,0,5212.50,752,20,0,2025-03-04
73832,5000,zzR9PvG7dY9u5iHU,6,0,9,0,4179.62,6,9,0,2025-03-19
25724,1000,zzXGgGu6ysGwGH1J,0,0,0,0,1000.00,6,0,0,2025-03-12


In [825]:
df_rams_recent = df_rams_recent.reset_index(drop=True)

## Adding New Variables

### Thoughts:

 - Higher OTB means they still have more money to spend, possibly indiciating that they will spend more in Q4
 - If spending increases throughout Q1 - Q3, spending might continue to increase in Q4
 - Higher current credit line means more money to spend

 


In [826]:
df_predict_Q4 = pd.merge(df_total_trans_Q1_to_Q3, df_rams_recent, on="current_account_nbr")
df_predict_Q4

,current_account_nbr,avg_trans_Q1_x,avg_trans_Q2_x,avg_trans_Q3_x,avg_trans_Q1_y,avg_trans_Q2_y,avg_trans_Q3_y,total_Q1,total_Q2,total_Q3,cu_crd_line,cu_bhv_scr,cu_nbr_days_dlq,ca_avg_utilz_lst_6_mnths,ca_nsf_count_lst_12_months,cu_otb,rb_new_bhv_scr,ca_avg_utilz_lst_3_mnths,ca_max_dlq_lst_6_mnths,cu_processing_date
0,033o9yHYen3xoz6k,1252.330000,306.939375,414.374615,32.135000,41.522842,48.115600,1380.87,8855.70,10198.43,15000,747,0,15,0,12599.88,744,14,0,2025-03-10
1,05AfSsd6epxXiK01,34.202500,383.900909,383.521064,29.842083,62.658500,110.187923,853.02,29423.34,38189.88,15000,740,0,39,0,11494.90,740,35,0,2025-02-27
2,05oLsFWaSLYzc5hv,45.220000,214.815000,106.415652,12.283333,23.003400,17.663810,209.36,5737.38,3931.32,15000,774,0,2,0,14881.28,774,0,0,2025-02-25
3,062wBe4JtO4ij7Mp,60.830000,412.893514,536.998750,70.965000,67.182686,85.497172,1317.93,27034.03,29581.05,25000,753,0,13,0,22154.83,752,12,0,2025-02-23
4,06SijOFpCXUvw9hs,2450.985000,555.494583,239.558800,49.036154,153.990196,45.690706,5539.44,21185.37,9872.68,25000,751,0,11,0,22454.40,754,12,0,2025-03-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,znzcg2AkVtchRVjt,35.355000,190.928276,203.155926,91.596667,57.197442,70.747222,345.50,7996.41,8032.11,15000,755,0,10,0,13210.05,755,11,0,2025-02-26
1495,zoBLVNjS9F2gDeYk,57.285000,507.469524,1049.958158,29.103750,97.974510,56.514945,813.06,51293.92,60469.85,25000,752,0,20,0,18325.42,730,16,0,2025-03-21
1496,zoD9wnDwZFJyCtcj,651.920000,132.946842,167.346667,13.318000,20.744333,16.279703,718.51,4392.98,5158.53,12800,760,0,6,0,11925.54,760,6,0,2025-03-21
1497,zsYIael1LtCzyZNe,21.582857,219.210400,180.819474,37.055000,151.780000,56.985263,447.52,8515.86,4518.29,14700,688,0,91,0,1183.89,694,92,0,2025-03-13


In [827]:
df_predict_Q4 = pd.merge(df_predict_Q4, df_trans_Q4_merged, on="current_account_nbr")
df_predict_Q4

,current_account_nbr,avg_trans_Q1_x,avg_trans_Q2_x,avg_trans_Q3_x,avg_trans_Q1_y,avg_trans_Q2_y,avg_trans_Q3_y,total_Q1,total_Q2,total_Q3,...,ca_avg_utilz_lst_6_mnths,ca_nsf_count_lst_12_months,cu_otb,rb_new_bhv_scr,ca_avg_utilz_lst_3_mnths,ca_max_dlq_lst_6_mnths,cu_processing_date,last_Q4_spending,avg_transaction_amt,avg_world_transaction_amt
0,033o9yHYen3xoz6k,1252.330000,306.939375,414.374615,32.135000,41.522842,48.115600,1380.87,8855.70,10198.43,...,15,0,12599.88,744,14,0,2025-03-10,14571.86,473.076471,46.639714
1,05AfSsd6epxXiK01,34.202500,383.900909,383.521064,29.842083,62.658500,110.187923,853.02,29423.34,38189.88,...,39,0,11494.90,740,35,0,2025-02-27,38729.78,370.094333,76.500556
2,05oLsFWaSLYzc5hv,45.220000,214.815000,106.415652,12.283333,23.003400,17.663810,209.36,5737.38,3931.32,...,2,0,14881.28,774,0,0,2025-02-25,3114.17,112.040000,58.224667
3,062wBe4JtO4ij7Mp,60.830000,412.893514,536.998750,70.965000,67.182686,85.497172,1317.93,27034.03,29581.05,...,13,0,22154.83,752,12,0,2025-02-23,23003.63,294.223721,61.254497
4,06SijOFpCXUvw9hs,2450.985000,555.494583,239.558800,49.036154,153.990196,45.690706,5539.44,21185.37,9872.68,...,11,0,22454.40,754,12,0,2025-03-16,12370.50,239.384815,46.149297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,znzcg2AkVtchRVjt,35.355000,190.928276,203.155926,91.596667,57.197442,70.747222,345.50,7996.41,8032.11,...,10,0,13210.05,755,11,0,2025-02-26,9306.84,185.269375,47.580563
1482,zoBLVNjS9F2gDeYk,57.285000,507.469524,1049.958158,29.103750,97.974510,56.514945,813.06,51293.92,60469.85,...,20,0,18325.42,730,16,0,2025-03-21,34958.43,415.649762,60.348759
1483,zoD9wnDwZFJyCtcj,651.920000,132.946842,167.346667,13.318000,20.744333,16.279703,718.51,4392.98,5158.53,...,6,0,11925.54,760,6,0,2025-03-21,4307.47,120.189231,12.715591
1484,zsYIael1LtCzyZNe,21.582857,219.210400,180.819474,37.055000,151.780000,56.985263,447.52,8515.86,4518.29,...,91,0,1183.89,694,92,0,2025-03-13,6123.16,162.772500,44.729429


In [828]:
cols_predict = ['total_Q1', 'total_Q2', 'total_Q3', 'last_Q4_spending', 'cu_crd_line', 'cu_otb', 'cu_bhv_scr', 'rb_new_bhv_scr', 'ca_avg_utilz_lst_6_mnths', 'ca_nsf_count_lst_12_months', 'ca_avg_utilz_lst_3_mnths', 'ca_max_dlq_lst_6_mnths']

In [829]:
df_predict_Q4 = df_predict_Q4[cols_predict]
df_predict_Q4

,total_Q1,total_Q2,total_Q3,last_Q4_spending,cu_crd_line,cu_otb,cu_bhv_scr,rb_new_bhv_scr,ca_avg_utilz_lst_6_mnths,ca_nsf_count_lst_12_months,ca_avg_utilz_lst_3_mnths,ca_max_dlq_lst_6_mnths
0,1380.87,8855.70,10198.43,14571.86,15000,12599.88,747,744,15,0,14,0
1,853.02,29423.34,38189.88,38729.78,15000,11494.90,740,740,39,0,35,0
2,209.36,5737.38,3931.32,3114.17,15000,14881.28,774,774,2,0,0,0
3,1317.93,27034.03,29581.05,23003.63,25000,22154.83,753,752,13,0,12,0
4,5539.44,21185.37,9872.68,12370.50,25000,22454.40,751,754,11,0,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1481,345.50,7996.41,8032.11,9306.84,15000,13210.05,755,755,10,0,11,0
1482,813.06,51293.92,60469.85,34958.43,25000,18325.42,752,730,20,0,16,0
1483,718.51,4392.98,5158.53,4307.47,12800,11925.54,760,760,6,0,6,0
1484,447.52,8515.86,4518.29,6123.16,14700,1183.89,688,694,91,0,92,0


## Finding Correlation

In [830]:
df_predict_Q4.corr(method="pearson")

,total_Q1,total_Q2,total_Q3,last_Q4_spending,cu_crd_line,cu_otb,cu_bhv_scr,rb_new_bhv_scr,ca_avg_utilz_lst_6_mnths,ca_nsf_count_lst_12_months,ca_avg_utilz_lst_3_mnths,ca_max_dlq_lst_6_mnths
total_Q1,1.000000,0.390531,0.378311,0.340146,0.101569,0.021213,-0.096426,-0.077111,0.110776,-0.012458,0.093485,0.078399
total_Q2,0.390531,1.000000,0.827516,0.768182,0.263622,0.074279,-0.109660,-0.080075,0.205253,0.006901,0.191892,-0.036731
total_Q3,0.378311,0.827516,1.000000,0.813836,0.264769,0.071633,-0.126868,-0.091822,0.228794,0.008749,0.206310,-0.048239
last_Q4_spending,0.340146,0.768182,0.813836,1.000000,0.265325,0.077028,-0.101478,-0.077039,0.227659,0.011237,0.180513,-0.066507
cu_crd_line,0.101569,0.263622,0.264769,0.265325,1.000000,0.848217,0.189103,0.186281,-0.211102,0.024689,-0.196079,-0.066676
cu_otb,0.021213,0.074279,0.071633,0.077028,0.848217,1.000000,0.392262,0.395129,-0.574210,0.030826,-0.588058,-0.095212
cu_bhv_scr,-0.096426,-0.109660,-0.126868,-0.101478,0.189103,0.392262,1.000000,0.780015,-0.621218,0.008492,-0.613154,-0.168894
rb_new_bhv_scr,-0.077111,-0.080075,-0.091822,-0.077039,0.186281,0.395129,0.780015,1.000000,-0.507820,0.016123,-0.507515,-0.159835
ca_avg_utilz_lst_6_mnths,0.110776,0.205253,0.228794,0.227659,-0.211102,-0.574210,-0.621218,-0.507820,1.000000,-0.016506,0.963405,0.105441
ca_nsf_count_lst_12_months,-0.012458,0.006901,0.008749,0.011237,0.024689,0.030826,0.008492,0.016123,-0.016506,1.000000,-0.013267,-0.013607


## Making Models

In [831]:
model_Q2 = smf.ols("total_Q2 ~ total_Q1 + cu_crd_line + ca_avg_utilz_lst_6_mnths", data = df_predict_Q4).fit()
model_Q2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               total_Q2   R-squared:                       0.251
Model:                            OLS   Adj. R-squared:                  0.250
Method:                 Least Squares   F-statistic:                     165.8
Date:                Sat, 29 Mar 2025   Prob (F-statistic):           1.17e-92
Time:                        18:11:41   Log-Likelihood:                -16072.
No. Observations:                1486   AIC:                         3.215e+04
Df Residuals:                    1482   BIC:                         3.217e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                 -529.9154   1145.139     -0.463      0.644   -2776.181    1716.350
total_Q1                     2.2399      0.151     14.790      0.000       1.943       2.537
cu_crd_line                  0.7415      0.062     11.953      0.000       0.620       0.863
ca_avg_utilz_lst_6_mnths   176.5857     18.104      9.754      0.000     141.073     212.098
==============================================================================
Omnibus:                     1083.674   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            38459.831
Skew:                           2.975   Prob(JB):                         0.00
Kurtosis:                      27.202   Cond. No.                     6.20e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.2e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [832]:
model_Q3 = smf.ols("total_Q3 ~ total_Q1 + total_Q2 + cu_crd_line + ca_avg_utilz_lst_6_mnths", data = df_predict_Q4).fit()
model_Q3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               total_Q3   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.695
Method:                 Least Squares   F-statistic:                     848.9
Date:                Sat, 29 Mar 2025   Prob (F-statistic):               0.00
Time:                        18:11:41   Log-Likelihood:                -15346.
No. Observations:                1486   AIC:                         3.070e+04
Df Residuals:                    1481   BIC:                         3.073e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                 -572.3951    702.763     -0.814      0.415   -1950.913     806.122
total_Q1                     0.3985      0.100      4.003      0.000       0.203       0.594
total_Q2                     0.7394      0.016     46.384      0.000       0.708       0.771
cu_crd_line                  0.1879      0.040      4.714      0.000       0.110       0.266
ca_avg_utilz_lst_6_mnths    59.9042     11.461      5.227      0.000      37.423      82.385
==============================================================================
Omnibus:                      268.623   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4389.981
Skew:                           0.321   Prob(JB):                         0.00
Kurtosis:                      11.396   Cond. No.                     9.82e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.82e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [833]:
model_Q4 = smf.ols("last_Q4_spending ~ total_Q1 + total_Q2 + total_Q3 + cu_crd_line + ca_avg_utilz_lst_6_mnths", data = df_predict_Q4).fit()
model_Q4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       last_Q4_spending   R-squared:                       0.695
Model:                            OLS   Adj. R-squared:                  0.694
Method:                 Least Squares   F-statistic:                     674.2
Date:                Sat, 29 Mar 2025   Prob (F-statistic):               0.00
Time:                        18:11:41   Log-Likelihood:                -15394.
No. Observations:                1486   AIC:                         3.080e+04
Df Residuals:                    1480   BIC:                         3.083e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                 -571.2733    726.275     -0.787      0.432   -1995.911     853.364
total_Q1                     0.0661      0.103      0.639      0.523      -0.137       0.269
total_Q2                     0.2853      0.026     11.059      0.000       0.235       0.336
total_Q3                     0.5616      0.027     20.917      0.000       0.509       0.614
cu_crd_line                  0.1482      0.041      3.571      0.000       0.067       0.230
ca_avg_utilz_lst_6_mnths    42.3197     11.950      3.541      0.000      18.879      65.761
==============================================================================
Omnibus:                      622.301   Durbin-Watson:                   2.164
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21969.641
Skew:                           1.281   Prob(JB):                         0.00
Kurtosis:                      21.662   Cond. No.                     1.24e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Getting Q1 2025

In [834]:
df_transactions_Q125 = df_transactions_clean[df_transactions_clean["transaction_date"].astype(str).str.startswith(("2025-01", "2025-02", "2025-03"))]
df_transactions_Q125

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
7,CMwVNwFlQtZ6lcWW,SALE,253,2025-02-02,70.78,NaN,0,1
8,0AqLx6QcUKlxYVnf,SALE,253,2025-02-14,158.97,NaN,0,1
9,cxK13NgvQWKwsdNH,PAYMENT,271,2025-03-11,200.00,NaN,0,0
10,0jk6WnqyQZIo0Bz8,SALE,253,2025-03-16,397.59,NaN,0,1
24,zhCQqBwTIUIFHnFH,SALE,253,2025-02-09,35.31,NaN,0,1
...,...,...,...,...,...,...,...,...
493330,xWPY5ARVAZvYDFOB,PAYMENT,271,2025-02-04,300.00,NaN,0,0
493331,ktpdvlbVRuywp2Po,SALE,253,2025-03-07,3.99,NaN,0,1
493333,zp60CmOQNComnHDT,SALE,253,2025-02-04,15.62,NaN,0,1
493334,8M5fnshTsgW9mW8Y,PAYMENT,271,2025-02-05,100.00,NaN,0,0


In [835]:
df_world_trans_Q125 = df_world_trans_clean[df_world_trans_clean["transaction_date"].astype(str).str.startswith(("2025-01", "2025-02", "2025-03"))]
df_world_trans_Q125

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt
6,ZTDQXJK90HWr8Qci,SALE,253,2025-01-19,5.07,NaN,0,1
7,SVLTisn540exAh2Y,SALE,253,2025-01-23,52.00,NaN,0,1
8,wQ2hfsT6Yb3svYu1,SALE,253,2025-02-11,10.00,NaN,0,1
18,ACScbBW7kMRJy2L2,SALE,253,2025-01-10,7.80,NaN,0,1
19,yV4tmHrHgEsgJkWb,SALE,253,2025-01-30,9.70,NaN,0,1
...,...,...,...,...,...,...,...,...
1053845,UCQDQq1PQEQicEll,SALE,253,2025-02-08,32.57,NaN,0,1
1053846,hTefCbHhFcf4yeoi,SALE,253,2025-02-20,5.04,NaN,0,1
1053848,RHjoQGhTFr6YvM5Z,SALE,253,2025-03-03,99.00,NaN,0,1
1053850,ZGjYc8aETjtytUYR,SALE,253,2025-03-10,16.11,NaN,0,1


In [836]:
df_transactions_Q125_sum = df_transactions_Q125.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_transactions_Q125_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
00gwujJkd597VxLd,25.90,25.900000
00iP5U82D8XwVQ9G,2714.50,16.653374
00oyr3QppAzjLws4,484.30,80.716667
02WRVV8yusgcZWLW,10440.58,1044.058000
033o9yHYen3xoz6k,5124.36,320.272500
...,...,...
zyZhjzJwhpOgSvmc,1632.96,233.280000
zz3nbtZXS41NZk0h,215.99,53.997500
zzBy2qNM78aRV580,440.00,220.000000


In [837]:
df_world_trans_Q125_sum = df_world_trans_Q125.groupby("current_account_nbr").agg(
    total_spent = ("transaction_amt", "sum"),
    avg_transaction_amt = ("transaction_amt", "mean")
)
df_world_trans_Q125_sum

,total_spent,avg_transaction_amt
current_account_nbr,,
02WRVV8yusgcZWLW,6503.30,103.226984
033o9yHYen3xoz6k,5997.06,50.822542
034bM166vNmgLiIA,221.24,24.582222
049sX01uVFhaa0i2,6415.82,50.123594
05AfSsd6epxXiK01,6592.47,59.931545
...,...,...
zuPY0iifqZT124F8,210.00,70.000000
zwtnyO2BGLGWUf7r,637.17,49.013077
zxOqFgWke0qPQZIO,0.00,0.000000


In [838]:
df_trans_Q125_merged = pd.merge(df_transactions_Q125_sum, df_world_trans_Q125_sum, on="current_account_nbr")
df_trans_Q125_merged

,total_spent_x,avg_transaction_amt_x,total_spent_y,avg_transaction_amt_y
current_account_nbr,,,,
02WRVV8yusgcZWLW,10440.58,1044.058000,6503.30,103.226984
033o9yHYen3xoz6k,5124.36,320.272500,5997.06,50.822542
034bM166vNmgLiIA,200.00,100.000000,221.24,24.582222
049sX01uVFhaa0i2,6404.95,800.618750,6415.82,50.123594
05AfSsd6epxXiK01,17934.55,498.181944,6592.47,59.931545
...,...,...,...,...
zsYIael1LtCzyZNe,4314.88,392.261818,420.80,24.752941
zwtnyO2BGLGWUf7r,5617.66,1872.553333,637.17,49.013077
zxOqFgWke0qPQZIO,383.58,191.790000,0.00,0.000000


In [839]:
df_trans_Q125_merged["total_Q125"] = df_trans_Q125_merged["total_spent_x"] + df_trans_Q125_merged["total_spent_y"]
df_trans_Q125_merged

,total_spent_x,avg_transaction_amt_x,total_spent_y,avg_transaction_amt_y,total_Q125
current_account_nbr,,,,,
02WRVV8yusgcZWLW,10440.58,1044.058000,6503.30,103.226984,16943.88
033o9yHYen3xoz6k,5124.36,320.272500,5997.06,50.822542,11121.42
034bM166vNmgLiIA,200.00,100.000000,221.24,24.582222,421.24
049sX01uVFhaa0i2,6404.95,800.618750,6415.82,50.123594,12820.77
05AfSsd6epxXiK01,17934.55,498.181944,6592.47,59.931545,24527.02
...,...,...,...,...,...
zsYIael1LtCzyZNe,4314.88,392.261818,420.80,24.752941,4735.68
zwtnyO2BGLGWUf7r,5617.66,1872.553333,637.17,49.013077,6254.83
zxOqFgWke0qPQZIO,383.58,191.790000,0.00,0.000000,383.58


In [840]:
df_trans_Q125_merged = df_trans_Q125_merged.drop(['total_spent_x', 'total_spent_y', 'avg_transaction_amt_x', 'avg_transaction_amt_y'], axis=1)
df_trans_Q125_merged

,total_Q125
current_account_nbr,
02WRVV8yusgcZWLW,16943.88
033o9yHYen3xoz6k,11121.42
034bM166vNmgLiIA,421.24
049sX01uVFhaa0i2,12820.77
05AfSsd6epxXiK01,24527.02
...,...
zsYIael1LtCzyZNe,4735.68
zwtnyO2BGLGWUf7r,6254.83
zxOqFgWke0qPQZIO,383.58


In [ ]:
def predict_Q2(Q1, credit_line, util_6):
    Q2_spending = -529.9154 + 2.2399*Q1 + 0.7415*credit_line + 176.5857*util_6
    return Q2_spending

In [842]:
def predict_Q3(Q1, Q2, credit_line, util_6):
    Q3_spending = -572.3951 + 0.3985*Q1 + 0.7394*Q2 + 0.1879*credit_line + 59.9042*util_6
    return Q3_spending

In [843]:
def predict_Q4(Q1, Q2, Q3, credit_line, util_6):
    Q4_spending = -571.2733 + 0.0661*Q1 + 0.2853*Q2 + 0.5616*Q3 + 0.1482*credit_line + 42.3197*util_6
    return Q4_spending